In [70]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import gym
from PIL import Image
import matplotlib.pyplot as plt
import random

In [71]:
env = gym.make('CartPole-v1', render_mode="rgb_array")

In [72]:
n_input = env.observation_space.shape[0] # 4
n_actions = env.action_space.n
n_output = n_actions

n_hidden1 = 100
n_hidden2 = 50

In [73]:
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_output])

In [74]:
weights = {
    'hidden1' : tf.Variable(tf.random_normal([n_input, n_hidden1], stddev=0.01)),
    'hidden2' : tf.Variable(tf.random_normal([n_hidden1, n_hidden2], stddev=0.01)),
    'output' : tf.Variable(tf.random_normal([n_hidden2, n_output], stddev=0.01))
}

biases = {
    'hidden1' : tf.Variable(tf.random_normal([n_hidden1], stddev=0.01)),
    'hidden2' : tf.Variable(tf.random_normal([n_hidden2], stddev=0.01)),
    'output' : tf.Variable(tf.random_normal([n_output], stddev=0.01))
}

In [75]:
def Qnet(x, weights, biases):
    hidden1 = tf.add(tf.matmul(x, weights['hidden1']), biases['hidden1'])
    hidden1 = tf.nn.relu(hidden1)

    hidden2 = tf.add(tf.matmul(hidden1, weights['hidden2']), biases['hidden2'])
    hidden2 = tf.nn.relu(hidden2)

    output = tf.add(tf.matmul(hidden2, weights['output']), biases['output'])
    return output

In [76]:
Qpred = Qnet(x, weights, biases)
loss = tf.reduce_mean(tf.square(y - Qpred))

LR = 0.001
optm = tf.train.AdamOptimizer(LR).minimize(loss)

In [94]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [95]:
gamma = 0.9

for episode in range(1001):
    terminated = False
    state = env.reset()[0]

    count = 0

    while not terminated:
        count += 1

        state = np.reshape(state, [1, n_input])
        Q = sess.run(Qpred, feed_dict={x: state})

        epsilon = 0.1
        if np.random.uniform() < epsilon:
            a = env.action_space.sample()
        else:
            a = np.argmax(Q)
        
        next_state, reward, terminated, _, _ = env.step(a)

        if terminated:
            reward = -200
            Q[0][a] = reward
        else:
            next_state = np.reshape(next_state, [1, n_input])
            next_Q = sess.run(Qpred, feed_dict={x: next_state})
            Q[0][a] = reward + gamma * np.max(next_Q)
        
        sess.run(optm, feed_dict={x: state, y: Q}) # Update

        state = next_state
    
    if episode % 100 == 0:
        print(f"Episode: {episode} steps: {count}")
env.close()

Episode: 0 steps: 8
Episode: 100 steps: 26
Episode: 200 steps: 27
Episode: 300 steps: 31
Episode: 400 steps: 59
Episode: 500 steps: 152
Episode: 600 steps: 84
Episode: 700 steps: 263
Episode: 800 steps: 173
Episode: 900 steps: 189
Episode: 1000 steps: 103


In [96]:
def show_gif(images: list) -> None:
    image_file = 'test.gif'
    images[0].save(image_file, save_all=True, append_images=images[1:], loop=0, duration=1)

state = env.reset()[0]
images = []

terminated = False

while not terminated:
    screen = env.render()
    images += [Image.fromarray(screen)]
    state = np.reshape(state, [1, n_input])
    Q = sess.run(Qpred, feed_dict={x: state})
    action = np.argmax(Q)

    next_state, reward, terminated, _, _ = env.step(action)
    state = next_state

env.close()

show_gif(images)
sess.close()

Action Replay

In [97]:
from collections import deque

In [98]:
# Settings
env = gym.make('CartPole-v1', render_mode="rgb_array")
n_input = env.observation_space.shape[0] # 4
n_actions = env.action_space.n
n_output = n_actions

n_hidden1 = 100
n_hidden2 = 50

x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_output])

In [99]:
weights = {
    'hidden1' : tf.Variable(tf.random_normal([n_input, n_hidden1], stddev=0.01)),
    'hidden2' : tf.Variable(tf.random_normal([n_hidden1, n_hidden2], stddev=0.01)),
    'output' : tf.Variable(tf.random_normal([n_hidden2, n_output], stddev=0.01))
}

biases = {
    'hidden1' : tf.Variable(tf.random_normal([n_hidden1], stddev=0.01)),
    'hidden2' : tf.Variable(tf.random_normal([n_hidden2], stddev=0.01)),
    'output' : tf.Variable(tf.random_normal([n_output], stddev=0.01))
}

In [100]:
def DQN(x, weights, biases):
    hidden1 = tf.add(tf.matmul(x, weights['hidden1']), biases['hidden1'])
    hidden1 = tf.nn.relu(hidden1)

    hidden2 = tf.add(tf.matmul(hidden1, weights['hidden2']), biases['hidden2'])
    hidden2 = tf.nn.relu(hidden2)

    output = tf.add(tf.matmul(hidden2, weights['output']), biases['output'])
    return output

In [161]:
Qpred = DQN(x, weights, biases)
loss = tf.reduce_mean(tf.square(y - Qpred))

LR = 0.0001
optm = tf.train.AdamOptimizer(LR).minimize(loss)

In [162]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)
replay_buffer = []
n_buffer = 5000
n_batch = 10
max_count = 0
gamma = 0.9

for episode in range(2001):
    terminated = False
    state = env.reset()[0]

    count = 0

    while not terminated:
        
        count += 1
        Q = sess.run(Qpred, feed_dict={x: np.reshape(state, [1, n_input])})

        epsilon = 0.1
        if np.random.uniform() < epsilon:
            a = env.action_space.sample()
        else:
            a = np.argmax(Q)
        
        next_state, reward, terminated, _, _ = env.step(a)

        if terminated:
            reward = -200
            # next_state = np.zeros(state.shape)

        replay_buffer.append([state, action, reward, next_state, terminated])
        if len(replay_buffer) > n_buffer:
            replay_buffer.pop(0)
        state = next_state
    max_count = max(count, max_count)
    if episode % n_batch == 1:
        for _ in range(100):
            if len(replay_buffer) < n_batch:
                continue
            mini_batch = random.sample(replay_buffer, n_batch)

            x_stack = np.empty(0).reshape(0, n_input)
            y_stack = np.empty(0).reshape(0, n_output)
            for state, action, reward, next_state, done in mini_batch:
                Q = sess.run(Qpred, feed_dict= {x: np.reshape(state, [1, n_input])})
                if terminated:
                    Q[0][action] = reward
                else:
                    next_Q = sess.run(Qpred, feed_dict= {x: np.reshape(next_state, [1, n_input])})
                    Q[0][action] = reward + gamma * np.max(next_Q)
                    
                x_stack = np.vstack([x_stack, state])
                y_stack = np.vstack([y_stack, Q])
            # Learning
            sess.run(optm, feed_dict={x: x_stack, y: y_stack}) # Update
    
    if episode % 200 == 0:
        print(f"Episode: {episode} steps: {count}, max count: {max_count}")
env.close()

Episode: 0 steps: 11, max count: 11
Episode: 200 steps: 9, max count: 15
Episode: 400 steps: 55, max count: 119
Episode: 600 steps: 35, max count: 210
Episode: 800 steps: 161, max count: 249
Episode: 1000 steps: 23, max count: 305
Episode: 1200 steps: 55, max count: 305
Episode: 1400 steps: 68, max count: 342
Episode: 1600 steps: 102, max count: 478
Episode: 1800 steps: 177, max count: 864
Episode: 2000 steps: 87, max count: 1340


In [163]:
state = env.reset()[0]
images = []

terminated = False

while not terminated:
    screen = env.render()
    images += [Image.fromarray(screen)]
    state = np.reshape(state, [1, n_input])
    Q = sess.run(Qpred, feed_dict={x: state})
    action = np.argmax(Q)

    next_state, reward, terminated, _, _ = env.step(action)
    state = next_state

env.close()

show_gif(images)

In [160]:
sess.close()